In [1]:
import numpy as np
import math
import collections
import random
from scipy.optimize import linprog
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
def hist(samp_list, min_s, max_s):
    h = [0] * (max_s - min_s + 1)
    for i in samp_list:
        h[i-min_s] += 1
    return h

def makeFingerPrint(samp_list):
    max_s = max(samp_list)
    min_s = min(samp_list)
    h1 = hist(samp_list, min_s, max_s)
    f = hist(h1, 0, max(h1))
    f = f[1:]
    return f

In [131]:
#FOR UNSEEN
def poisspdf(x, l):
    v1 = math.pow(l, x)
    v2 = math.factorial(x)
    v3 = math.pow(math.e, -l)
    final = (v1 / v2) * v3
    return final

def getSampleSize(f):
    k = 0
    for i,entry in enumerate(f):
        k = k + entry * (i+1)
    return k

#check
def getFirstNonZeroIndex(f):
    for i,entry in enumerate(f):
        if(entry != 0):
            return i+1

#check
def getLastNonZeroIndex(f):
    for i, entry in reversed(list(enumerate(f))):
        if(entry != 0):
            return i+1
        return -1

def getProbabilityMass(x, histx):
    pmass = 0
    for i, x_entry in enumerate(x):
        pmass = pmass + x_entry * histx[i]
    return pmass

def init_XLP(xLPmax, xLPmin, gridFactor):
    max_pow = math.ceil(math.log(xLPmax/xLPmin) / math.log(gridFactor))
    xLP = []
    for i in range(0, max_pow+1):
        xLP.append(xLPmin * math.pow(gridFactor, i))
    return xLP

def init_objf(szLPx, szLPf, fLP):
    objf = [0] * (szLPx + 2*szLPf)
    j = 0
    for i in range(szLPx, len(objf), 2):
        objf[i] = 1 / math.sqrt(fLP[j] + 1)
        j = j + 1
    j = 0
    for i in range(szLPx+1, len(objf), 2):
        objf[i] = 1 / math.sqrt(fLP[j] + 1)
        j = j + 1
    return objf

def init_A_b(szLPf, szLPx, xLP, fLP, k):
    #A = [[0] * (szLPx + 2 * szLPf)] * 2*szLPf
    A = []
    for i in range(2*szLPf):
        tmp = []
        for j in range(szLPx + 2 * szLPf):
            tmp.append(0)
        A.append(tmp)
    
    b = [0] * (2*szLPf)
    for i in range (0, szLPf):
        for j in range(0, szLPx):
            A[2 * i][j] = poisspdf(i+1, k * xLP[j])
        for k in range(0, szLPx):
            A[2*i+1][k] = -1 * A[2*i][k]
        A[2 * i][szLPx + 2 * i] = -1
        A[(2 * i)  + 1][szLPx + (2 * i)  + 1] = -1
        b[2 * i] = fLP[i]
        b[(2 * i)  + 1] = -fLP[i]
    
    return A,b

def rescale_cond(A, Aeq, xLP, szLPx):
    for i in range(0, szLPx):
        for j in range(0, len(A)):
            A[j][i] = A[j][i]/xLP[i]
        Aeq[i] = Aeq[i]/xLP[i]
    return A, Aeq

    
def unseen(f):
    k = getSampleSize(f)
    
    gridFactor = 1.05
    alpha = 0.5
    
    xLPmin = 1 / (k * max(10,k))
    
    min_i = getFirstNonZeroIndex(f) #Returning index + 1
    
    if min_i > 1:
        xLPmin = min_i/k
    
    maxLPIters = 1000
    x = [0]
    histx = [0]
    
    fLP = [0] * len(f)
    for i in range(0, len(f)):
        i_m = i+1
        if(f[i] > 0):
            wind = [max(1, i_m - math.ceil(math.sqrt(i_m))), 
                    min(i_m + math.ceil(math.sqrt(i_m)), len(f))]
            
            low_ind = wind[0] - 1
            high_ind = wind[1]
            sum_f = sum(f[low_ind : high_ind])
            if( sum_f < math.sqrt(i_m)):
                x.append(i_m/k)
                histx.append(f[i])
                fLP[i] = 0
            else:
                fLP[i] = f[i]
    
    #If no LP portion, retun the empirical histogram
    fmax = getLastNonZeroIndex(f)
    if(fmax == -1):
        x.pop(0)
        histx.pop(0)
        return [x, histx]
    
    #Setting up first LP
    LPmass = 1 - getProbabilityMass(x, histx)
    
    z_flp = [0] * math.ceil(math.sqrt(fmax))
    fLP.extend(z_flp)
    
    szLPf = len(fLP)
    
    xLPmax = fmax/k
    
    xLP = init_XLP(xLPmax, xLPmin, gridFactor)
    
    szLPx = len(xLP)
    
    objf = init_objf(szLPx, szLPf, fLP)
    
    A, b = init_A_b(szLPf, szLPx, xLP, fLP, k)
    
    Aeq = [0] * (szLPx+2*szLPf)
    for i in range(0,szLPx):
        Aeq[i] = xLP[i]
    beq = LPmass
    
    A, Aeq = rescale_cond(A, Aeq, xLP, szLPx)
    
    options = {'maxiter':1000, 'disp':False}
    lb = [0] * (szLPx+2*szLPf)
    ub = [float('Inf')] * (szLPx+2*szLPf)
    
    res = linprog(objf, A, b, [Aeq], beq, list(zip(lb, ub)), options={'disp':False}, method='interior-point')
    
    #Second LP
    objf2 = [0] * len(objf)
    for i in range(0, szLPx):
        objf2[i] = 1
    
    A2 = []
    for row in A:
        A2.append(row)
    A2.append(objf)
    
    b.append(res['fun']+alpha)
    b2 = b
    for i in range(0, szLPx):
        objf2[i] = objf2[i]/xLP[i]
    
    res = linprog(objf2, A2, b2, [Aeq], beq, list(zip(lb, ub)), options={'disp':False}, method='interior-point')
    
    sol2 = res['x']
    
    for i in range (0, szLPx):
        sol2[i] = sol2[i] / xLP[i]
    
    for i in range (0, len(xLP)):
        x.append(xLP[i])
    for i in range (0, len(sol2)):
        histx.append(sol2[i])
    
    ind = np.argsort(x)
    x.sort()
    
    tempHistX = []
    for i in range (0, len(ind)):
        tempHistX.append(histx[ind[i]])
    histx = tempHistX
    
    ind = []
    for i in range (0, len(histx)):
        if histx[i] > 0:
            ind.append(i)
            
    tempX = []
    if x is not None:
        for i in range (0, len(ind)):
            tempX.append(x[ind[i]])

        x = tempX
    
    
    tempHistX = []
    for i in range (0, len(ind)):
        tempHistX.append(histx[ind[i]])
    histx = tempHistX
    
    return histx, x

In [132]:
def entropy_estC(f):
    k = getSampleSize(f)
    
    gridFactor = 1.05
    alpha = 0.5
    
    xLPmin = 1 / (k * max(10,k))
    
    min_i = getFirstNonZeroIndex(f) #Returning index + 1
    
    if min_i > 1:
        xLPmin = min_i/k
    
    maxLPIters = 1000
    x = [0]
    histx = [0]
    
    fLP = [0] * len(f)
    for i in range(0, len(f)):
        i_m = i+1
        if(f[i] > 0):
            wind = [max(1, i_m - math.ceil(math.sqrt(i_m))), 
                    min(i_m + math.ceil(math.sqrt(i_m)), len(f))]
            
            
            low_ind = wind[0] - 1
            high_ind = wind[1]
            sum_f = sum(f[low_ind : high_ind])
            if( sum_f < math.sqrt(i_m)):
                x.append(i_m/k)
                histx.append(f[i])
                fLP[i] = 0
            else:
                fLP[i] = f[i]
    
    #If no LP portion, retun the empirical histogram and corrected empirical entropy
    fmax = getLastNonZeroIndex(f)
    if(fmax == -1):
        x.pop(0)
        histx.pop(0)
        sumHistX = 0
        for j in range (0, len(histx)):
            sumHistX += histx[j]
        logX = 0
        for j in range (0, len(x)):
            logxX += log(x[j]) * x[j] * histx[j]
        ent = -logX + sumHistX / (2 * k)
        return ent
    
    #Setting up first LP
    LPmass = 1 - getProbabilityMass(x, histx)
    
    z_flp = [0] * math.ceil(math.sqrt(fmax))
    fLP.extend(z_flp)
    
    szLPf = len(fLP)
    
    xLPmax = fmax/k
    
    xLP = init_XLP(xLPmax, xLPmin, gridFactor)
    
    szLPx = len(xLP)
    
    objf = init_objf(szLPx, szLPf, fLP)
    
    A, b = init_A_b(szLPf, szLPx, xLP, fLP, k)
    
    Aeq = [0] * (szLPx+2*szLPf)
    for i in range(0,szLPx):
        Aeq[i] = xLP[i]
    beq = LPmass
    
    A, Aeq = rescale_cond(A, Aeq, xLP, szLPx)
    
    options = {'maxiter':1000, 'disp':False}
    lb = [0] * (szLPx+2*szLPf)
    ub = [float('Inf')] * (szLPx+2*szLPf)
    
    res = linprog(objf, A, b, [Aeq], beq, list(zip(lb, ub)), options={'disp':False}, method='interior-point')
    sol = res['x']
    
    #Second LP
    if min_i < 2:
        objf2 = [0] * len(objf)
        for i in range(0, szLPx):
            objf2[i] = 1

        A2 = []
        for row in A:
            A2.append(row)
        A2.append(objf)

        b.append(res['fun']+alpha)
        b2 = b
        for i in range(0, szLPx):
            objf2[i] = objf2[i]/xLP[i]

        res = linprog(objf2, A2, b2, [Aeq], beq, list(zip(lb, ub)), options={'disp':False}, method='interior-point')
    
        sol2 = res['x']
    else:
        sol2 = sol
    
    for i in range (0, szLPx):
        sol2[i] = sol2[i] / xLP[i]
    
    # append LP solution to empirical portion of histogram

    firstNonZeroIndex = -1
    # find first non-zero index
    for i in range(0, len(f)):
        if(f[i] > 0):
            firstNonZeroIndex = i
            break
    ent = 0
    if firstNonZeroIndex == -1:
        # check
        for j in range (0, szLPx):
            ent += sol2[j] * -1 * xlp[j] * math.log(xLP[j])
    else:
        positiveXIndices = []
        for j in range (0, len(x)):
            if x[j] > 0:
                positiveXIndices.append(j)
                
        histXPositiveSum = 0
        histXPositive = []
        for j in range (0, len(positiveXIndices)):
            histXPositive.append(histx[positiveXIndices[j]])
            histXPositiveSum += histx[positiveXIndices[j]]
        
        xPositive = []
        for j in range (0, len(positiveXIndices)):
            xPositive.append(x[positiveXIndices[j]])
            
        histXLogXProduct = 0
        for j in range (0, len(histXPositive)):
            histXLogXProduct -= histXPositive[j] * xPositive[j] * math.log(xPositive[j])
            
        sol2XLPProcut = 0
        for j in range (0, szLPx):
            sol2XLPProcut += sol2[j] * xLP[j] * math.log(xLP[j])
        
        ent = histXLogXProduct + histXPositiveSum / (2 * k) - sol2XLPProcut
    
    
    for i in range (0, len(xLP)):
        x.append(xLP[i])
    for i in range (0, len(sol2)):
        histx.append(sol2[i])
    
    ind = np.argsort(x)
    x.sort()
    
    tempHistX = []
    for i in range (0, len(ind)):
        tempHistX.append(histx[ind[i]])
    histx = tempHistX
    
    ind = []
    for i in range (0, len(histx)):
        if histx[i] > 0:
            ind.append(i)
            
    if x is not None:
        tempX = []
        for i in range (0, len(ind)):
            tempX.append(x[ind[i]])

        x = tempX
    
    tempHistX = []
    for i in range (0, len(ind)):
        tempHistX.append(histx[ind[i]])
    histx = tempHistX
    
    return ent

In [115]:
import math

def getRMSE(trueEntropy, estimatedEntropyList):
    squareDiffSum = 0
    for estimatedEntropy in estimatedEntropyList:
        squareDiffSum += ((estimatedEntropy - trueEntropy) ** 2)
    return math.sqrt(squareDiffSum / len(estimatedEntropyList))

In [181]:
%%time

nList = []
kList = []
rmseList = []

for k in [1000]:
    #for n in range(int(k / 10), k * 10, int((k * 10 - k / 10) / 10)):
    for n in [500, 1000, 2000, 3000, 4000, 5000, 6000, 8000, 10000]:

        nList.append(n)
        kList.append(k)
        
        estimatedEntropyList = []
        for z in range(0, 100):
            
            sample = []
            for x in range(0, int(k)):
                sample.append(int(random.randint(1, n)))
            f = makeFingerPrint(sample)
            h, x = unseen(f)
#             print("True Entropy = ")
            #output entropy of the true distribution, Unif[n]
            trueEntropy = math.log(n)
#             print(trueEntropy)

#             print("Empirical Entropy = ")
            #output entropy of the empirical distribution of the sample
            empiricalEntropy = 0
            for i in range (0, len(f)):
                empiricalEntropy -= f[i] * (i + 1) / k * math.log((i + 1) / k)
#             print(empiricalEntropy)

#             print("Estimated Entropy = ")
            #output entropy of the recovered histogram, [h,x]
            estimatedEntropy = 0
            if x is not None:
                for i in range(0, len(x)):
                    estimatedEntropy -= h[i] * x[i] * math.log(x[i])
#             print(estimatedEntropy)

            e2 = entropy_estC(f)
#             print("====================================================================")

#             print(trueEntropy)
#             print(empiricalEntropy)
#             print(estimatedEntropy)
#             print(e2)
            estimatedEntropyList.append(e2)
            
        rmseList.append(getRMSE(trueEntropy, estimatedEntropyList))
        #print(n,k)
        #print(estimatedEntropyList)

/usr/local/lib/python3.7/site-packages/scipy/optimize/_linprog_ip.py:1262: OptimizeWarning:

Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.

/usr/local/lib/python3.7/site-packages/scipy/optimize/_linprog_ip.py:1274: OptimizeWarning:

Solving system with option 'sym_pos':False failed. This may happen occasionally, especially as the solution is approached. However, if you see this frequently, your problem may be numerically challenging. If you cannot improve the formulation, consider setting 'lstsq' to True. Consider also setting `presolve` to True, if it is not already.



CPU times: user 9min 48s, sys: 54.7 s, total: 10min 43s
Wall time: 4min 18s


In [182]:
import pandas as pd

df = pd.DataFrame()
df['k'] = kList
df['n'] = nList
df['rmse'] = rmseList


In [183]:
df_10 = df[df['k'] == 1000]
df_500 = df[df['k'] == 10000]
df_1000 = df[df['k'] == 20000]

In [184]:
df_10

,k,n,rmse
0,1000,500,0.696900
1,1000,1000,0.305935
2,1000,2000,0.098476
3,1000,3000,0.169929
4,1000,4000,0.176798
5,1000,5000,0.158825
6,1000,6000,0.127782
7,1000,8000,0.128687
8,1000,10000,0.149025


In [157]:
df_1000

,k,n,rmse
20,20000,2000,0.499711
21,20000,21800,2315.379641
22,20000,41600,2505.925041
23,20000,61400,2160.471579
24,20000,81200,2588.796313
25,20000,101000,2779.043370
26,20000,120800,2490.825098
27,20000,140600,2430.028342
28,20000,160400,2004.762678
29,20000,180200,2241.965224


In [48]:
df_1000.head()

,k,n,rmse
20,20000,2000,0.432326
21,20000,21800,2337.540335
22,20000,41600,2531.503200
23,20000,61400,2107.599002
24,20000,81200,2650.796833


In [189]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

trace0 = go.Scatter(
    x = df_10['n'],
    y = df_10['rmse'],
    mode = 'lines',
    name = 'k = 10'
)
trace1 = go.Scatter(
    x = df_500['n'],
    y = df_500['rmse'],
    mode = 'lines',
    name = 'k = 500'
)
trace2 = go.Scatter(
    x = df_1000['n'],
    y = df_1000['rmse'],
    mode = 'lines',
    name = 'k = 1000'
)

data = [trace0]
layout = go.Layout(
#     xaxis=dict(
#         type='log',
#         autorange=True
#     ),
    yaxis=dict(
#         tickmode='linear',
#         ticks='outside',
#         range=[0, 1]
#         dtick=0.25,
#         ticklen=8,
#         tickwidth=4,
#         tickcolor='#000'
#         type='log',
#         autorange=True
#         ticklen=8
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='scatter-mode')

In [191]:
%%time

nList = []
kList = []
rmseList = []

for k in [10000]:
    #for n in range(int(k / 10), k * 10, int((k * 10 - k / 10) / 10)):
    for n in [1000, 10000, 20000, 30000, 40000, 50000, 60000, 80000, 100000]:

        nList.append(n)
        kList.append(k)
        
        estimatedEntropyList = []
        for z in range(0, 100):
            
            sample = []
            for x in range(0, int(k)):
                sample.append(int(random.randint(1, n)))
            f = makeFingerPrint(sample)
            h, x = unseen(f)
#             print("True Entropy = ")
            #output entropy of the true distribution, Unif[n]
            trueEntropy = math.log(n)
#             print(trueEntropy)

#             print("Empirical Entropy = ")
            #output entropy of the empirical distribution of the sample
            empiricalEntropy = 0
            for i in range (0, len(f)):
                empiricalEntropy -= f[i] * (i + 1) / k * math.log((i + 1) / k)
#             print(empiricalEntropy)

#             print("Estimated Entropy = ")
            #output entropy of the recovered histogram, [h,x]
            estimatedEntropy = 0
            if x is not None:
                for i in range(0, len(x)):
                    estimatedEntropy -= h[i] * x[i] * math.log(x[i])
#             print(estimatedEntropy)

            e2 = entropy_estC(f)
#             print("====================================================================")

#             print(trueEntropy)
#             print(empiricalEntropy)
#             print(estimatedEntropy)
#             print(e2)
            estimatedEntropyList.append(e2)
            
        rmseList.append(getRMSE(trueEntropy, estimatedEntropyList))
        #print(n,k)
        #print(estimatedEntropyList)

/usr/local/lib/python3.7/site-packages/scipy/optimize/_linprog_ip.py:1262: OptimizeWarning:

Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.

/usr/local/lib/python3.7/site-packages/scipy/optimize/_linprog_ip.py:1274: OptimizeWarning:

Solving system with option 'sym_pos':False failed. This may happen occasionally, especially as the solution is approached. However, if you see this frequently, your problem may be numerically challenging. If you cannot improve the formulation, consider setting 'lstsq' to True. Consider also setting `presolve` to True, if it is not already.



CPU times: user 1min 58s, sys: 5.56 s, total: 2min 4s
Wall time: 1min 26s


In [192]:
import pandas as pd

df = pd.DataFrame()
df['k'] = kList
df['n'] = nList
df['rmse'] = rmseList

In [193]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

trace0 = go.Scatter(
    x = df['n'],
    y = df['rmse'],
    mode = 'lines',
    name = 'k = 10000'
)
# trace1 = go.Scatter(
#     x = df_500['n'],
#     y = df_500['rmse'],
#     mode = 'lines',
#     name = 'k = 500'
# )
# trace2 = go.Scatter(
#     x = df_1000['n'],
#     y = df_1000['rmse'],
#     mode = 'lines',
#     name = 'k = 1000'
# )

data = [trace0]
layout = go.Layout(
#     xaxis=dict(
#         type='log',
#         autorange=True
#     ),
    yaxis=dict(
#         tickmode='linear',
#         ticks='outside',
#         range=[0, 1]
#         dtick=0.25,
#         ticklen=8,
#         tickwidth=4,
#         tickcolor='#000'
#         type='log',
#         autorange=True
#         ticklen=8
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='scatter-mode')

In [17]:
df.to_csv("rmse_2.csv")

In [100]:
n = 10000
k = 1000
f = [904,45,2]
h, x = unseen(f)
print("True Entropy = ")
#output entropy of the true distribution, Unif[n]
trueEntropy = math.log(n)
print(trueEntropy)

print("Empirical Entropy = ")
#output entropy of the empirical distribution of the sample
empiricalEntropy = 0
for i in range (0, len(f)):
    empiricalEntropy -= f[i] * (i + 1) / k * math.log((i + 1) / k)
print(empiricalEntropy)

print("Estimated Entropy = ")
#output entropy of the recovered histogram, [h,x]
estimatedEntropy = 0
if x is not None:
    for i in range(0, len(x)):
        estimatedEntropy -= h[i] * x[i] * math.log(x[i])
print(estimatedEntropy)

e2 = entropy_estC(f)
print("====================================================================")

print(trueEntropy)
print(empiricalEntropy)
print(estimatedEntropy)
print(e2)

True Entropy = 
9.210340371976184
Empirical Entropy = 
6.838780358999733
Estimated Entropy = 
11.607355903010351
9.210340371976184
6.838780358999733
11.607355903010351
11.607377215109963
